In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Load and Preprocess the Data

First load the two data sets and check them out.

In [2]:
MSA1 = pd.read_excel("./raw_data/MSA1.xlsx", 
                     sheet_name="Property Status", 
                     skiprows=4)
MSA2 = pd.read_excel("./raw_data/MSA2.xlsx", 
                     sheet_name="Property Status", 
                     skiprows=4)

In [3]:
print(MSA1.shape)
print(MSA2.shape)

(108, 154)
(858, 154)


In [4]:
MSA1.head()

,MarketCode,MarketName,StateCode,StateName,CountyCode,CountyName,TractCode,BlockGroupCode,Latitude,Longitude,...,Jan-18,Feb-18,Mar-18,Apr-18,May-18,Jun-18,Jul-18,Aug-18,Sep-18,Oct-18
0,10420,"Akron, OH",OH,Ohio,153,Summit,530104.0,4.0,41.31810,-81.44502,...,S,S,S,S,S,S,S,S,S,S
1,10420,"Akron, OH",OH,Ohio,153,Summit,502102.0,1.0,41.10346,-81.46938,...,S,S,S,S,S,S,S,S,S,S
2,10420,"Akron, OH",OH,Ohio,153,Summit,530603.0,1.0,41.17920,-81.48354,...,S,S,S,S,S,S,S,S,S,S
3,10420,"Akron, OH",OH,Ohio,153,Summit,530401.0,1.0,41.17894,-81.43911,...,S,S,S,S,S,S,S,S,S,S
4,10420,"Akron, OH",OH,Ohio,153,Summit,NaN,NaN,41.10510,-81.46834,...,S,S,S,S,S,S,S,S,S,S


In [5]:
MSA2.head()

,MarketCode,MarketName,StateCode,StateName,CountyCode,CountyName,TractCode,BlockGroupCode,Latitude,Longitude,...,Jan-18,Feb-18,Mar-18,Apr-18,May-18,Jun-18,Jul-18,Aug-18,Sep-18,Oct-18
0,12420,"Austin-Round Rock, TX",TX,Texas,453,Travis,NaN,NaN,30.30917,-97.72881,...,S,S,S,S,S,S,S,S,S,S
1,12420,"Austin-Round Rock, TX",TX,Texas,209,Hays,010400,3.0,29.84896,-97.94931,...,UC/LU,UC/LU,UC/LU,UC/LU,UC/LU,UC/LU,UC/LU,UC/LU,UC/LU,LU
2,12420,"Austin-Round Rock, TX",TX,Texas,453,Travis,001860,2.0,30.45009,-97.64873,...,S,S,S,S,S,S,S,S,S,S
3,12420,"Austin-Round Rock, TX",TX,Texas,453,Travis,000604,3.0,30.28462,-97.74723,...,S,S,S,S,S,S,S,S,S,S
4,12420,"Austin-Round Rock, TX",TX,Texas,453,Travis,NaN,NaN,30.28147,-97.75801,...,S,S,S,S,S,S,S,S,S,S


We can assert that each asset is represented by its unique ProjID in every dataframe.

In [6]:
assert len((MSA1.ProjID).unique()) == len(MSA1)
assert len((MSA2.ProjID).unique()) == len(MSA2)

Therefore, for this problem, we only need the ProjID, and the Property Status columns overtime. We can define the following function to preprocess the raw data frame and get the new dataframe with the columns that we want to study.

In [7]:
def preprocess(MSA):
    """Return a dataframe after preprocess"""
    
    # Use regex to get all the time columns
    MSA_times = MSA.filter(regex=("\-"))
    MSA_id = MSA[['ProjID', 'Status']]
    
    MSA_new = pd.concat([MSA_id, MSA_times], axis = 1)
    
    # We only want to calculate the assets with a current status of 'S'
    MSA_new = MSA_new[MSA_new.Status == 'S']
    
    MSA_new.set_index('ProjID', inplace=True)
    MSA_new.drop(columns=['Status'], inplace=True)
    return MSA_new

In [8]:
MSA1_new = preprocess(MSA1)
MSA2_new = preprocess(MSA2)

In [9]:
MSA1_new.head()

,Apr-08,May-08,Jun-08,Jul-08,Aug-08,Sep-08,Oct-08,Nov-08,Dec-08,Jan-09,...,Jan-18,Feb-18,Mar-18,Apr-18,May-18,Jun-18,Jul-18,Aug-18,Sep-18,Oct-18
ProjID,,,,,,,,,,,,,,,,,,,,,
13558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,S,S,S,S,S,S,S,S,S
77380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,S,S,S,S,S,S,S,S,S
17397,S,S,S,S,S,S,S,S,S,S,...,S,S,S,S,S,S,S,S,S,S
67674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,S,S,S,S,S,S,S,S,S
10050488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,S,S,S,S,S,S,S,S,S


In [10]:
MSA2_new.head()

,Apr-08,May-08,Jun-08,Jul-08,Aug-08,Sep-08,Oct-08,Nov-08,Dec-08,Jan-09,...,Jan-18,Feb-18,Mar-18,Apr-18,May-18,Jun-18,Jul-18,Aug-18,Sep-18,Oct-18
ProjID,,,,,,,,,,,,,,,,,,,,,
10031809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,S,S,S,S,S,S,S,S,S
89519,S,S,S,S,S,S,S,S,S,S,...,S,S,S,S,S,S,S,S,S,S
43056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,S,S,S,S,S,S,S,S,S
10029492,S,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,S,S,S,S,S,S,S,S,S,S
10018608,S,S,S,S,S,S,S,S,S,S,...,S,S,S,S,S,S,S,S,S,S


## Question 1

For this question, I will define a function with boolean return value to check whether the first status of that asset is 'S' or 'R'. Then, I will apply this function to each of the dataframes by row.

In [11]:
def check_first_status(row):
    row = row.dropna()
    if (row[0] == 'S') or (row[0] == 'R'):
        return False
    else:
        return True

In [12]:
MSA1_new['first_status'] = MSA1_new.apply(check_first_status, axis = 1)
result1 = len(MSA1_new[MSA1_new.first_status == True])
print("There are "+str(result1)+" properties delivered in MSA1.")

There are 10 properties delivered in MSA1.


In [13]:
MSA2_new['first_status'] = MSA2_new.apply(check_first_status, axis = 1)
result2 = len(MSA2_new[MSA2_new.first_status == True])
print("There are "+str(result2)+" properties delivered in MSA2.")

There are 214 properties delivered in MSA2.


As we can see from the results above, since April 2008, there are 10 properties delivered in the Ohio market (MSA1) and 214 properties delivered in the Texas market (MSA2).

## Question 2

For this question, I will define a function that returns the lease-up time measured in month for each property. And I will apply this function by row to the dataframes that only contains properties delivered after April 2008.

In [14]:
def lease_up_time(row):
    row = row.dropna()
    if (row[0] == 'S') or (row[0] == 'R'):
        return 0
    for i in range(len(row)):
        if row[i] == 'S':
            return i

In [15]:
MSA1_ns = MSA1_new[MSA1_new.first_status == True]
MSA1_ns = MSA1_ns.drop(columns=['first_status'])
rs1 = np.mean(MSA1_ns.apply(lease_up_time, axis = 1))
print("The average lease-up time is "+str(rs1)+" months in MSA1.")

The average lease-up time is 7.3 months in MSA1.


In [16]:
MSA2_ns = MSA2_new[MSA2_new.first_status == True]
MSA2_ns = MSA2_ns.drop(columns=['first_status'])
rs2 = np.mean(MSA2_ns.apply(lease_up_time, axis = 1))
print("The average lease-up time is "+str(rs2)+" months in MSA2.")

The average lease-up time is 10.69626168224299 months in MSA2.


As we can see from the results above, the average lease-up time in the Ohio market (MSA1) is 7.3 months and is 10.7 months in the Texas market (MSA2).